Media Cloud: Measuring Attention
================================
- Reference: https://github.com/mediacloud/api-tutorial-notebooks/blob/main/MC02%20-%20attention.ipynb
- Media Ids Query: https://search.mediacloud.org/sources/1

In [54]:
# Set up your API key and import needed things
import os, mediacloud.api
from importlib.metadata import version

# from dotenv import load_dotenv
import datetime as dt
from IPython.display import JSON, display
import bokeh.io
from rich import print as pp
import json
from tqdm import tqdm
import pandas as pd

tqdm.pandas()
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False, nb_workers=8, verbose=0)
bokeh.io.reset_output()
bokeh.io.output_notebook(hide_banner=True)

MC_API_KEY = os.getenv("MEDIA_CLOUD_API_KEY")
if MC_API_KEY is None:
    try:
        with open("../config/media.cloud.key") as f:
            MC_API_KEY = f.read().strip()
        pp("[bold green][SUCCESS] MC API Key found.[/bold green]")
    except FileNotFoundError:
        pp(
            "[bold red][ERROR] MC API key not found. Check ENV 'MEDIA_CLOUD_API_KEY' or file './config/media.cloud.key'[/bold red]"
        )
else:
    pp("[bold green][SUCCESS] MC API Key found.[/bold green]")
search_api = mediacloud.api.SearchApi(MC_API_KEY)
# pp(f"[gray][INFO] Using Media Cloud python client v{version('mediacloud')}[/gray]")

[SUCCESS] MC API Key found.

## Listing Stories

Story counts are fine, but often what you really want is the story themselves. Note that **we cannot provide story content** due to copyright restrictions. However, you can get a list of all the URLs and then fetch them yourself. We can also return word counts down to the story level (see the "language" notebook for more info on that).

In [53]:
from email.mime import base
import requests


def qury_media_id(media_name: str = "nytimes"):
    try:
        response = requests.get(
            url="https://search.mediacloud.org/api/sources/sources/",
            params={
                "limit": "3",
                "name": media_name,
            },
            headers={
                "Cookie": "csrftoken=qqQVYfPizRLDITAtYCMn4ShmotfrK69T; sessionid=nk9ykemged6ukcvfjgfsg63p8l93p7ra",
            },
        )
        if response.status_code == 200:
            # print(response.json())
            return response.json()["results"]
        else:
            pp(f"[red]ERROR: {response.status_code}[/red]")
    except requests.exceptions.RequestException:
        pp("[red]HTTP Request failed[/red]")


# test
qury_media_id("nytimes")[0]

{'id': 1,
 'name': 'nytimes.com',
 'url_search_string': '',
 'label': 'New York Times',
 'homepage': 'http://nytimes.com',
 'notes': None,
 'platform': 'online_news',
 'stories_per_week': 1245,
 'first_story': None,
 'created_at': '2022-12-23T17:43:28.547804Z',
 'modified_at': '2024-08-31T13:43:19.039389Z',
 'pub_country': 'USA',
 'pub_state': 'US-NY',
 'primary_language': None,
 'media_type': 'print_native',
 'collection_count': 45}

In [100]:
df_media_id = pd.read_csv(
    "/home/rongxin/data/sync/trump-exposure/scripts/media-cloud-search/data/meta/Collection-34412234-United States - National-sources-20240907062857.csv"
).drop(
    columns=[
        "url_search_string",
        "notes",
        "platform",
        "first_story",
        "pub_country",
        "media_type",
        "pub_state",
    ]
)
df_media_id["primary_language"] = df_media_id["primary_language"].fillna("")
df_media_id["name_clean"] = df_media_id["name"].parallel_apply(
    lambda x: x.lower().split(".")[0]
)

name_mapping = {
    "abcnews": "abc",
    "cbsnews": "cbs",
    "theguardian": "guardian",
    "huffingtonpost": "hp",
    "nypost": "nyp",
    "nytimes": "nyt",
    "usatoday": "usa",
    "washingtonpost": "wp",
    "breitbart": "bb",
    "businessinsider": "bi",
    "dailycaller": "caller",
    "foxnews": "fox",
    "nbcnews": "nbc",
    "politico": "pol",
}
df_media_id["name_clean"] = df_media_id["name"].parallel_apply(
    lambda x: x.lower().split(".")[0]
)
df_media_id["name_clean"] = df_media_id["name_clean"].replace(name_mapping)
df_media_id["stories_per_week"] = df_media_id["stories_per_week"].fillna(-1).astype(int)
df_media_id.to_csv(
    "../data/meta/Collection-34412234-United States - National-sources-20240907062857-cleaned.csv",
    index=False,
)
df_media_id.head()

,id,name,label,homepage,stories_per_week,primary_language,name_clean
0,1,nytimes.com,New York Times,http://nytimes.com,1245,,nyt
1,2,washingtonpost.com,Washington Post,http://washingtonpost.com,1940,,wp
2,3,csmonitor.com,Christian Science Monitor,http://csmonitor.com,93,en,csmonitor
3,4,usatoday.com,USA Today,http://www.usatoday.com,1422,,usa
4,6,latimes.com,LA Times,http://www.latimes.com/,1310,,latimes


In [101]:
df_media_ideo = pd.read_csv(
    "/home/rongxin/data/sync/trump-exposure/scripts/media-cloud-search/data/ideology/media_ideo.csv"
)[["media", "medianame", "ideo"]]
df_media_ideo["name_clean"] = df_media_ideo["media"].parallel_apply(
    lambda x: x.lower().split("u_")[1]
)
df_media_ideo.head()

,media,medianame,ideo,name_clean
0,u_abc,ABC,2.931940,abc
1,u_bb,Breitbart,4.069811,bb
2,u_bbc,BBC,2.498193,bbc
3,u_bi,Business Insider,2.652033,bi
4,u_buzz,BuzzFeed,2.497727,buzz


In [102]:
df_media_id = (
    pd.merge(df_media_id, df_media_ideo, on="name_clean", how="outer")
    .dropna()
    .reset_index(drop=True)
)
df_media_id["id"] = df_media_id["id"].astype(int)
df_media_id

,id,name,label,homepage,stories_per_week,primary_language,name_clean,media,medianame,ideo
0,19260,abcnews.go.com,abcnews.go.com,http://abcnews.go.com,1017.0,,abc,u_abc,ABC,2.931940
1,19334,breitbart.com,Breitbart,http://www.breitbart.com,667.0,,bb,u_bb,Breitbart,4.069811
2,18710,businessinsider.com,Business Insider,http://www.businessinsider.com,186.0,en,bi,u_bi,Business Insider,2.652033
3,18775,dailycaller.com,Daily Caller,http://dailycaller.com,651.0,,caller,u_caller,Daily Caller,3.926641
4,1752,cbsnews.com,CBS News,http://www.cbsnews.com/,744.0,,cbs,u_cbs,CBS,2.893285
5,1095,cnn.com,CNN,http://www.cnn.com/,1175.0,,cnn,u_cnn,CNN,2.666406
6,1092,foxnews.com,Fox News,http://www.foxnews.com/,1446.0,,fox,u_fox,Fox,3.577958
7,314,huffingtonpost.com,The Huffington Post (broken),http://www.huffingtonpost.com/,1399.0,en,hp,u_hp,HuffPost,2.361066
8,293951,msnbc.com,msnbc.com 1,http://msnbc.com/,124.0,en,msnbc,u_msnbc,MSNBC,2.597879
9,25499,nbcnews.com,NBC News,http://www.nbcnews.com,200.0,en,nbc,u_nbc,NBC,2.825059


In [87]:
# let's fetch all the stories matching our query on one day
my_query = '"trump"'  # note the double quotes used to indicate use of the whole phrase
start_date = dt.date(2023, 11, 1)
end_date = dt.date(2023, 11, 2)
sources = [1, 4, 111803, 897236]
all_stories = []
US_NATIONAL_COLLECTION = 34412234
more_stories = True
pagination_token = None

while more_stories:
    page, pagination_token = search_api.story_list(
        my_query,
        start_date,
        end_date,
        # collection_ids=[US_NATIONAL_COLLECTION],
        # source_ids=sources,
        source_ids=df_media_id["id"].tolist(),
        pagination_token=pagination_token,
    )
    all_stories += page
    more_stories = pagination_token is not None


def clean_story_dates(story):
    story["publish_date"] = str(story["publish_date"].strftime("%Y-%m-%d"))
    story["indexed_date"] = str(story["indexed_date"].strftime("%Y-%m-%d"))
    return json.dumps(story)


all_stories = [clean_story_dates(story) for story in all_stories]
pp(f"[bold green]Found {len(all_stories)} stories[/bold green]")
all_stories

Found 197 stories

['{"id": "85fb7df5412912edca11b777f3bb2db87c8aaa288fb027c79b719cd2b41e05e3", "media_name": "abcnews.go.com", "media_url": "abcnews.go.com", "title": "Donald Trump Jr. to testify in Trump Organization\'s $250 million fraud trial", "publish_date": "2023-11-01", "url": "https://abcnews.go.com/US/donald-trump-jr-testify-trump-organizations-250-million/story?id=104527860", "language": "en", "indexed_date": "2024-02-17"}',
 '{"id": "1bdc420d5fe2af96fbe1ddf5af9642297083219ec8a3d25b0a647e11a3d4c10d", "media_name": "abcnews.go.com", "media_url": "abcnews.go.com", "title": "Video Microsoft launches AI work assistant", "publish_date": "2023-11-01", "url": "https://abcnews.go.com/Technology/video/microsoft-launches-ai-work-assistant-104532624", "language": "en", "indexed_date": "2024-02-17"}',
 '{"id": "600a7137dae89667609727866bdd61e1f832a034d60412bf30dabcf6f6723f60", "media_name": "wsj.com", "media_url": "wsj.com", "title": "Trump Disqualification Challenges Face Tests in Two States", "publish_d

In [52]:
"nytimes" in str(all_stories)

False

### Writing a CSV of Story Data

What you probably want is a csv of all this story data. Here's a quick exmaple of dumping that data to a CSV (like our Search tool does).

In [ ]:
import csv

fieldnames = [
    "id",
    "publish_date",
    "title",
    "url",
    "language",
    "media_name",
    "media_url",
    "indexed_date",
]
with open("story-list.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction="ignore")
    writer.writeheader()
    for s in all_stories:
        writer.writerow(s)

In [ ]:
# and let's make sure it worked by checking out by loading it up as a pandas DataFrame
import pandas

df = pandas.read_csv("story-list.csv")
df.head()